<a href="https://colab.research.google.com/github/manjunathsirur1994/DataCleaning-EDA/blob/main/Pipelines_and_composite_estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines and composite estimators

## Pipeline chaining estimators

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [ ]:
# defining a pipeline
estimator = [('reduce_dim',PCA()), ('clf',SVC())]
pipe = Pipeline(estimator)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

In [ ]:
# Lets look at the stages of pipeline

pipe[0]

PCA()

In [ ]:
pipe[1]

SVC()

## Tracking feature names in pipeline

In [ ]:
# how to look at featuers with pipeline

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
import pandas as pd

In [ ]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [ ]:
x = df.drop(columns='target')
y = df['target']

In [ ]:
estimator = [('select_k_best_parameters',SelectKBest()),('logistic_regression',LogisticRegression())]
pipe = Pipeline(estimator)
pipe

Pipeline(steps=[('select_k_best_parameters', SelectKBest()),
                ('logistic_regression', LogisticRegression())])

In [ ]:
pipe.fit(x, y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:776: UserWarning: k=10 is greater than n_features=4. All the features will be returned.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('select_k_best_parameters', SelectKBest()),
                ('logistic_regression', LogisticRegression())])

In [ ]:
# Navigate to select_k_best_parameters and .get_feature_names_out()

print(f'lets see whats in pipe[:-1]: {pipe[:-1]}')
print(pipe[:-1].get_feature_names_out())

lets see whats in pipe[:-1]: Pipeline(steps=[('select_k_best_parameters', SelectKBest())])
['sepal length (cm)' 'sepal width (cm)' 'petal length (cm)'
 'petal width (cm)']


In [ ]:
# using grid search cv in pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV

param_grid = dict(reduce_dim=['passthrough', PCA(n_components=2), PCA(n_components=5), PCA(n_components=10)],
                  clf = [SVC(), LogisticRegression()],
                  clf__C=[1, 10, 20, 50, 100])

grid_search = GridSearchCV(pipe, param_grid=param_grid)

### Lets implement pipeline with grid search cv end to end [along with scalers and dimensionality reduction]

standard scaler

pca with n_components

classifers: logistic regression vs support vector classifier

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [ ]:
x,y = load_digits(return_X_y=True)

In [ ]:
x

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [ ]:
y

array([0, 1, 2, ..., 8, 9, 8])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('dim_reduction','passthrough'),
    ('classifiers','passthrough')
])

param_grid = [
    {
        'dim_reduction':[PCA()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[SVC()],
        'classifiers__C':[0.1,1,10],
        'classifiers__kernel':['linear','rbf']
    },
    {
        'dim_reduction':[NMF()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[SVC()],
        'classifiers__C':[0.1,1,10],
        'classifiers__kernel':['linear','rbf']
    },
    {
        'dim_reduction':[PCA()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[LogisticRegression(max_iter=1000)],
        'classifiers__C':[0.01,0.1,1,10]
    },
    {
        'dim_reduction':[NMF()],
        'dim_reduction__n_components':[2,5,8,10],
        'classifiers':[LogisticRegression()],
        'classifiers__C':[0.01,0.1,1,10]
    }
]

grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(x,y)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
400 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packages/

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('dim_reduction', 'passthrough'),
                                       ('classifiers', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'classifiers': [SVC()],
                          'classifiers__C': [0.1, 1, 10],
                          'classifiers__kernel': ['linear', 'rbf'],
                          'dim_reduction': [PCA()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [SVC()],
                          'classifier...
                          'dim_reduction': [NMF()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [LogisticRegression(max_iter=1000)],
                          'classifiers__C': [0.01, 0.1, 1, 10],
                          'dim_reduction': [PCA()],
                          'dim_reduction__n_components': [2, 5, 8, 10]},
                         {'classifiers': [LogisticRegression()],
                          'classifiers__C': [0.01, 0.1, 1, 10],
                          'dim_reduction': [NMF()],
                          'dim_reduction__n_components': [2, 5, 8, 10]}],
             scoring='accuracy')

In [ ]:
# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Parameters: {'classifiers': SVC(), 'classifiers__C': 10, 'classifiers__kernel': 'rbf', 'dim_reduction': PCA(), 'dim_reduction__n_components': 10}
Best Accuracy: 0.9326815642458101


In [ ]:
pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('dim_reduction',PCA(n_components=10)),
    ('classifier',SVC(C=10, kernel='rbf'))
])
pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=10)),
                ('classifier', SVC(C=10))])

In [ ]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('dim_reduction', PCA(n_components=10)),
                ('classifier', SVC(C=10))])

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9555555555555556

## Transforming target in Regression
TransformedTargetRegressor transforms the targets y before fitting a regression model. The predictions are mapped back to the original space via an inverse transform. It takes as an argument the regressor that will be used for prediction, and the transformer that will be applied to the target variable

In [37]:
from sklearn.datasets import fetch_california_housing
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [46]:
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [47]:
x = df.drop(columns='target')
y = df['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [48]:
transformer = QuantileTransformer(output_distribution='normal')
regressor = LinearRegression()
regressor_transformer = TransformedTargetRegressor(regressor=regressor, transformer=transformer)

In [49]:
pipe = Pipeline([
    # ("scaler", StandardScaler()),
    # ('pca', 'passthrough'),
    ('regressor', 'passthrough')
])

param_grid = [
    {
        # 'pca':[PCA()],
        # 'pca__n_components':[2,5,8,10],
        'regressor':[regressor_transformer]
    }
]

grid_search_cv = GridSearchCV(pipe, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_cv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=Pipeline(steps=[('regressor', 'passthrough')]),
             n_jobs=-1,
             param_grid=[{'regressor': [TransformedTargetRegressor(regressor=LinearRegression(),
                                                                   transformer=QuantileTransformer(output_distribution='normal'))]}],
             scoring='neg_mean_squared_error')

In [50]:
# Best parameters and score
print("Best Parameters:", grid_search_cv.best_params_)
print("Best Accuracy:", grid_search_cv.best_score_)

Best Parameters: {'regressor': TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=QuantileTransformer(output_distribution='normal'))}
Best Accuracy: -0.5854380166246823


In [51]:
# lets see the accuracy of the linear_regression model without target transformation

model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
model.score(x_test, y_test)

0.614222395017058